In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [29]:
import os
from langchain_ollama import ChatOllama

model_name="granite4:1b"
model_url=os.getenv('OLLAMA_HOST')

model = ChatOllama(
    model=model_name,
    api_base=model_url
)

In [30]:
from langchain.tools import tool, ToolRuntime

@tool
def read_email(runtime: ToolRuntime) -> str:
    """Read an email from the given address."""
    # take email from state
    return runtime.state["email"]

@tool
def send_email(body: str) -> str:
    """Send an email to the given address with the given subject and body."""
    # fake email sending
    return f"Email sent"

In [31]:
from langchain.agents import create_agent, AgentState
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.middleware import HumanInTheLoopMiddleware

class EmailState(AgentState):
    email: str

agent = create_agent(
    model=model,
    tools=[read_email, send_email],
    state_schema=EmailState,
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "read_email": False,
                "send_email": True,
            },
            description_prefix="Tool execution requires approval",
        ),
    ],
)

In [32]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {
        "messages": [HumanMessage(content="Please read my email and send a response.")],
        "email": "Hi Seán, I'm going to be late for our meeting tomorrow. Can we reschedule? Best, John."
    },
    config=config
)

In [12]:
from pprint import pprint

pprint(response)

{'__interrupt__': [Interrupt(value={'action_requests': [{'args': {'body': 'Hi '
                                                                          'Seán, '
                                                                          "I'm "
                                                                          'going '
                                                                          'to '
                                                                          'be '
                                                                          'late '
                                                                          'for '
                                                                          'our '
                                                                          'meeting '
                                                                          'tomorrow. '
                                                                          'Can '
                

In [33]:
print(response['__interrupt__'])

[Interrupt(value={'action_requests': [{'name': 'send_email', 'args': {'body': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we reschedule? Best, John."}, 'description': 'Tool execution requires approval\n\nTool: send_email\nArgs: {\'body\': "Hi Seán, I\'m going to be late for our meeting tomorrow. Can we reschedule? Best, John."}'}], 'review_configs': [{'action_name': 'send_email', 'allowed_decisions': ['approve', 'edit', 'reject']}]}, id='0b116f230991573d10fed6df6e33e7b4')]


In [34]:
# Access just the 'body' argument from the tool call
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

Hi Seán, I'm going to be late for our meeting tomorrow. Can we reschedule? Best, John.


## Approve

In [15]:
from langgraph.types import Command

response = agent.invoke(
    Command( 
        resume={"decisions": [{"type": "approve"}]}
    ), 
    config=config # Same thread ID to resume the paused conversation
)

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response.', additional_kwargs={}, response_metadata={}, id='69cb6678-e5ca-4ffa-b4cc-9b8ac9b6816d'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-27T16:45:13.0949015Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5478321000, 'load_duration': 4434834600, 'prompt_eval_count': 214, 'prompt_eval_duration': 384164700, 'eval_count': 16, 'eval_duration': 604732800, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019b60b3-0caa-75b2-99f9-6031ec67c6a6-0', tool_calls=[{'name': 'read_email', 'args': {}, 'id': 'ec6bad24-c3f3-4749-9164-38a60d671a9a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 214, 'output_tokens': 16, 'total_tokens': 230}),
              ToolMessage(content="Hi Seán,

## Reject

In [39]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "2"}}

response = agent.invoke(
    {
        "messages": [HumanMessage(content="Please read my email and send a response.")],
        "email": "Hi Seán, I'm going to be late for our meeting tomorrow. Can we reschedule? Best, John."
    },
    config=config
)

In [40]:
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "reject",
                    # An explanation of why the request was rejected
                    "message": "No please sign off - Your merciful leader, Seán."
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response.', additional_kwargs={}, response_metadata={}, id='e821181d-b494-4546-9425-73bd3c31a06e'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-27T16:52:29.6474001Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1111293100, 'load_duration': 91529400, 'prompt_eval_count': 214, 'prompt_eval_duration': 369372800, 'eval_count': 16, 'eval_duration': 542376100, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019b60b9-c701-7482-9bd3-eaf3794b2db0-0', tool_calls=[{'name': 'read_email', 'args': {}, 'id': 'e3aff398-f6d3-432f-b713-73cdf8c7d078', 'type': 'tool_call'}], usage_metadata={'input_tokens': 214, 'output_tokens': 16, 'total_tokens': 230}),
              ToolMessage(content="Hi Seán, I

In [36]:
print(response['__interrupt__'][0].value['action_requests'][0]['args']['body'])

KeyError: '__interrupt__'

## Edit

In [38]:
response = agent.invoke(
    Command(        
        resume={
            "decisions": [
                {
                    "type": "edit",
                    # Edited action with tool name and args
                    "edited_action": {
                        # Tool name to call.
                        # Will usually be the same as the original action.
                        "name": "send_email",
                        # Arguments to pass to the tool.
                        "args": {"body": "This is the last straw, you're fired!"},
                    }
                }
            ]
        }
    ), 
    config=config # Same thread ID to resume the paused conversation
    )   

pprint(response)

{'email': "Hi Seán, I'm going to be late for our meeting tomorrow. Can we "
          'reschedule? Best, John.',
 'messages': [HumanMessage(content='Please read my email and send a response.', additional_kwargs={}, response_metadata={}, id='a0277d5d-1e67-42bf-a088-a0fe3935ff92'),
              AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'granite4:1b', 'created_at': '2025-12-27T16:48:48.4451502Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1026226000, 'load_duration': 80723600, 'prompt_eval_count': 214, 'prompt_eval_duration': 296869900, 'eval_count': 16, 'eval_duration': 553008400, 'logprobs': None, 'model_name': 'granite4:1b', 'model_provider': 'ollama'}, id='lc_run--019b60b6-6755-70a2-845d-5f94c9c41012-0', tool_calls=[{'name': 'read_email', 'args': {}, 'id': '8a5b072a-1359-4fde-8df3-2ded06351532', 'type': 'tool_call'}], usage_metadata={'input_tokens': 214, 'output_tokens': 16, 'total_tokens': 230}),
              ToolMessage(content="Hi Seán, I